# 逆行列


$n$次正方行列$A$に対し

$$
A^{-1}A = AA^{-1} = I
$$

を満たす正方行列$A^{-1}$が存在するとき、$A$は **正則行列（regular matrix）** や **可逆行列（invertible matrix）** とよばれる。

また、$A^{-1}$を$A$の**逆行列**という。（$I$は$A$と同じサイズの単位行列）

正則でない行列は **特異行列（singular matrix）** という。

### 正則行列の条件

正方行列$A$が正則行列であるための必要十分条件は$\det(A) \neq 0$である。

$A$の逆行列$A^{-1}$の計算方法の一つに

$$
A^{-1} = \frac{1}{ \det(A) } \tilde{A}
$$

があり、分母が$\det(A) \neq 0$である必要がある。ここで$\tilde{A}$は$A$の余因子行列である。

In [1]:
import numpy as np

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0],
])

def cofactor(A, i, j):
    """余因子を計算する関数"""
    A_minor = np.delete(A, i, axis=0)
    A_minor = np.delete(A_minor, j, axis=1)
    return (-1)**(i + j) * np.linalg.det(A_minor)

def cofactor_matrix(A):
    """余因子行列"""
    C = np.zeros_like(A, dtype=np.float32)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            C[i, j] = cofactor(A, i=i, j=j)
    return C.T

# det(A) != 0 かどうかを確認
print(f"det(A) = {np.linalg.det(A)}")

# 余因子行列の計算
A_adj = cofactor_matrix(A)
print(f"\n余因子行列 A_adj:\n{A_adj}")

# 逆行列
A_inv = (1 / np.linalg.det(A)) * A_adj
print(f"\n逆行列 A^-1:\n{A_inv.round(3)}")

det(A) = 27.0

余因子行列 A_adj:
[[-48.  24.  -3.]
 [ 42. -21.   6.]
 [ -3.   6.  -3.]]

逆行列 A^-1:
[[-1.778  0.889 -0.111]
 [ 1.556 -0.778  0.222]
 [-0.111  0.222 -0.111]]


In [18]:
# ちゃんと逆行列になっているか確認
(A_inv @ A).round(1)

array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [ 0.,  0.,  1.]])

### 余因子行列と逆行列

$A$を$n$次正方行列、$\tilde{A}$を$A$の余因子行列、$I$を$n$次単位行列とすると

$$
A \tilde{A}
= \tilde{A} A
= \det(A) I
$$

という関係がある。

なので、$A$が正則であれば

$$
\underbrace{ \frac{1}{\det(A)} \tilde{A} }_{A^{-1}} A = I
$$

となる。

In [24]:
print(f"det(A) = {np.linalg.det(A):.0f}")
print(f"A_adj @ A: \n{A_adj @ A}")

det(A) = 27
A_adj @ A: 
[[27.  0.  0.]
 [ 0. 27.  0.]
 [ 0.  0. 27.]]


## 逆行列の計算規則

$$
(A^{-1})^{-1} = A
$$

$$
(AB)^{-1} = B^{-1} A^{-1}
$$

$$
(A^k)^{-1} = (A^{-1})^k
$$

In [2]:
import numpy as np

A = np.array([
    [1, 2],
    [3, 4],
])
B = A + 3
A_inv = np.linalg.inv(A)

A_inv @ B @ A

array([[-11., -16.],
       [ 15.,  22.]])

## 逆行列の効率的な計算

### 行列式を使う場合

$$
A^{-1} = \frac{1}{|A|} \tilde{A}
$$

から計算する場合、おそらく$O((n-1)^3 \times n^2)$かかる

**行列式の計算量**

$$
|A| = \sum_{\sigma \in S_n} \operatorname{sgn}(\sigma) a_{1 \sigma(1)} a_{2 \sigma(2)} \cdots a_{n \sigma(n)}
$$

という行列式の定義から、置換の集合$S_n$の要素数が$n!$個あり、各項で$n$回$a_{n \sigma(n)}$を掛けるので$O(n\times n!)$

「行列Aのi行から、j行の定数倍を引いても、行列式の値は変わらない。」という性質を使って上三角行列に変形してから行列式を計算する。三角行列への変換は$O(n^3)$、上三角行列の行列式は対角成分の積なので$O(n)$なので、全体で$O(n^3)$になる

（参考：[Tech Tips: 行列式の計算](https://techtipshoge.blogspot.com/2011/08/blog-post_23.html)）


**余因子行列の計算量**

$(i, j)$余因子は「『$i$行目と$j$列目を除いた行列』の行列式に$(-1)^{i+j}$をかけたもの」なので、$n-1$次行列の行列式を求める計算になり、$O((n-1)^3)$

余因子行列は「$(i, j)$余因子を$i,j$成分に持つ行列を転置したもの」なので、$(i, j)$余因子を$n\times n$個計算する必要がある→$O((n-1)^3 \times n^2)$か？


### 掃き出し法による計算

教科書的な方法でありながら、行列分解による方法と計算量が大差ない（=結構効率的）


### 行列分解で計算する方法

- https://speakerdeck.com/nagiss/hiyurisuteitukukontesutodeji-jie-xue-xi-siyou?slide=26

### 共役勾配法

連立一次方程式

$$
Ax = b
$$

の$x$を求めることは、二次形式

$$
f(x) = \frac{1}{2} x^T Ax - b^T x
$$

の最小化と一致するので、$f(x)$の最小値を探索する方法で解を求める方法を **共役勾配（conjugate gradient: CG）法** という

[連立1次方程式：共役勾配法 - PukiWiki for PBCG Lab](https://www.slis.tsukuba.ac.jp/~fujisawa.makoto.fu/cgi-bin/wiki/index.php?%CF%A2%CE%A91%BC%A1%CA%FD%C4%F8%BC%B0%A1%A7%B6%A6%CC%F2%B8%FB%C7%DB%CB%A1)

:::{margin}
**逆行列計算と連立一次方程式の関連性**

行列での連立一次方程式

$$
AX = I
$$

を考えると、この解は$X=A^{-1}$になる
:::